Genero
---
>V de Cramer - Todo o conjunto

>Q de Yule - Pares

Idade
---
>Kruskal Gamma - Todo o conjunto


>Kruskal Gamma - Pares

IMC
---
>Kruskal Gamma - Todo o conjunto

>Kruskal Gamma - Pares

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, norm
import math

from pathlib import Path

import researchpy
#from scipy.spatial.distance import yule

In [2]:
# Definição de diretórios e Disco de leitura dos arquivos (Desktop ou Notebook) 
config_dirs = open("prefixo_dados.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip())

DIRETORIO_PRINCIPAL =  DIR_PREFIXO / "Projeto_PA" / "Projeto_PA_validado"

DIR_BASE = DIRETORIO_PRINCIPAL / "analise_abordagens"

### Funções de medidas de associações: Q de Yule, V Cramer e Kruskal Gamma

In [3]:
def comb(n,k):
    return int(math.factorial(n) / (math.factorial(k)*math.factorial(n - k)))

def yule(m):
    m = np.array(m)
    a = ((m[0][0]*m[1][1]) - (m[0][1]*m[1][0]))
    b = ((m[0][0]*m[1][1]) + (m[0][1]*m[1][0]))
    if b == 0:
        y = 0.0
    else:
        y = np.round(a / b, 4)


    if y < 0:
        return y, 'Associação inversa'
    else:
        if y >= 0.0 and y <= 0.2:
             return y, 'Associação direta - ausente'
        elif y > 0.2 and y <= 0.5:
             return y, 'Associação direta - pequena'
        elif y > 0.5 and y <= 0.8:
             return y, 'Associação direta - moderada'
        elif y > 0.8:
            return y, 'Associação direta - grande'

#Referência: ausente: [0-0,2]; pequena: (0,2-0,5]; moderada: (0,5-0,8]; grande: > 0,8.","\n",
def cramer_v(m):
    if min(m.shape) < 2:
        print('\n\nDados com dimensão menor que 2!\n\n')
        return 0
    
    try:
        chi2, p, dof, ex = chi2_contingency(m)
    except:
        chi2 = 0
        
    v = np.round((chi2/(m.sum().sum() * (min(m.shape) - 1))) ** (0.5), 4)
    
    if v <= 0.2:
        return v, 'Associação Ausente'
    elif v <= 0.5:
        return v, 'Associação Pequena'
    elif v <= 0.8:
        return v, 'Associação Moderada'
    elif v > 0.8:
        return v, 'Associação Grande'
    
def kruskal_g(data):#, ordinal1, ordinal2, orderLabels1, orderLabels2=None):
# myCrosstable = pd.crosstab(data[ordinal1], data[ordinal2])
# 
# myCrosstable = myCrosstable.reindex(orderLabels1)
#         
# if orderLabels2 == None:
#     myCrosstable = myCrosstable[orderLabels1]
# else:
#     myCrosstable = myCrosstable[orderLabels2]
    myCrosstable = data
    nRows = myCrosstable.shape[0]
    nCols = myCrosstable.shape[1]
    
    
    C = [[0 for x in range(nCols)] for y in range(nRows)] 

    # top left part
    for i in range(nRows):
        for j in range(nCols):
            h = i-1
            k = j-1        
            if h>=0 and k>=0:            
                for p in range(h+1):
                    for q in range(k+1):
                        C[i][j] = C[i][j] + list(myCrosstable.iloc[p])[q]

    # bottom right part                    
    for i in range(nRows):
        for j in range(nCols):
            h = i+1
            k = j+1        
            if h<nRows and k<nCols:            
                for p in range(h, nRows):
                    for q in range(k, nCols):
                        C[i][j] = C[i][j] + list(myCrosstable.iloc[p])[q]
                        
    D = [[0 for x in range(nCols)] for y in range(nRows)] 

    # bottom left part
    for i in range(nRows):
        for j in range(nCols):
            h = i+1
            k = j-1        
            if h<nRows and k>=0:            
                for p in range(h, nRows):
                    for q in range(k+1):
                        D[i][j] = D[i][j] + list(myCrosstable.iloc[p])[q]

    # top right part                    
    for i in range(nRows):
        for j in range(nCols):
            h = i-1
            k = j+1        
            if h>=0 and k<nCols:            
                for p in range(h+1):
                    for q in range(k, nCols):
                        D[i][j] = D[i][j] + list(myCrosstable.iloc[p])[q]

    P = 0
    Q = 0
    for i in range(nRows):
        for j in range(nCols):
            P = P + C[i][j] * list(myCrosstable.iloc[i])[j]
            Q = Q + D[i][j] * list(myCrosstable.iloc[i])[j]
               
    GKgamma = (P - Q) / (P + Q)
    
#    if abs(GKgamma) < .10:
#        qual = 'Negligible'
#    elif abs(GKgamma) < .20:
#        qual = 'Weak'
#    elif abs(GKgamma) < .40:
#        qual = 'Moderate'
#    elif abs(GKgamma) < .60:
#        qual = 'Relatively strong'
#    elif abs(GKgamma) < .80:
#        qual = 'Strong'        
#    else:
#        qual = 'Very strong'

    if GKgamma < 0:
        qual = 'Associação Inversa'
    elif GKgamma <= .20:
        qual = 'Associação Direta - Ausente'
    elif GKgamma <= .50:
        qual = 'Associação Direta - Pequena'
    elif GKgamma <= .80:
        qual = 'Associação Direta - Moderada'
    elif GKgamma > .80:
        qual = 'Associação Direta - Grande'        

#    n = myCrosstable.sum().sum()
#    
#    Z1 = GKgamma * ((P + Q) / (n * (1 - GKgamma**2)))**0.5
#    
#    forASE0 = 0
#    forASE1 = 0
#    for i in range(nRows):
#        for j in range(nCols):
#            forASE0 = forASE0 + list(myCrosstable.iloc[i])[j] * (Q * C[i][j] - P * D[i][j])**2
#            forASE1 = forASE1 + list(myCrosstable.iloc[i])[j] * (C[i][j] - D[i][j])**2
#
#    ASE0 = 4 * (forASE0)**0.5 / (P + Q)**2
#    ASE1 = 2 * (forASE1 - (P - Q)**2 / n)**0.5 / (P + Q)        
#    Z2 = GKgamma / ASE0
#    Z3 = GKgamma / ASE1
#    
#    p1 = norm.sf(Z1)
#    p2 = norm.sf(Z2)
#    p3 = norm.sf(Z3)
#    
#    zvalues = [Z1] + [Z2] + [Z3]
#    pvalues = [p1] + [p2] + [p3]
            
    return (np.round(GKgamma, 4),qual), 0 #zvalues, pvalues


In [4]:
def aplica_cramer(df, df_saida):
    n_linhas = comb(df.shape[1],2) + 1
    n_colunas = df.shape[1]
    
    lista_cramer_quant = list()
    lista_cramer_quali = list()
    
    resultado = cramer_v(df)
    
    lista_cramer_quant.append (resultado[0])
    lista_cramer_quali.append (resultado[1])
    
    for i in range(n_linhas - 1):
        lista_cramer_quant.append(np.nan)
        lista_cramer_quali.append(np.nan)
    
    df_saida['V de Cramer (Quant)'] = lista_cramer_quant
    df_saida['V de Cramer - Quali'] = lista_cramer_quali
    
    return df_saida

def aplica_yule(df, df_saida):
    #n_linhas = comb(df.shape[1],2) + 1
    n_colunas = df.shape[1]
    
    lista_yule_quant = list()
    lista_yule_quali = list()

    lista_yule_quant.append(np.nan)
    lista_yule_quali.append(np.nan)

    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):

            resultado = yule(df[[f'cluster {cluster1}', f'cluster {cluster2}']])

            lista_yule_quant.append(resultado[0])
            lista_yule_quali.append(resultado[1])

    df_saida['Q de Yule - Quantitativo'] = lista_yule_quant
    df_saida['Q de Yule - Qualitativo'] = lista_yule_quali
    
    return df_saida

def aplica_kruskal(df, df_saida):
    #n_linhas = comb(df.shape[1],2) + 1
    n_colunas = df.shape[1]
    
    lista_kruskal_quant = list()
    lista_kruskal_quali = list()

    resultado = kruskal_g(df)[0]

    lista_kruskal_quant.append(resultado[0])
    lista_kruskal_quali.append(resultado[1])

    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):
            resultado = kruskal_g(df[[f'cluster {cluster1}', f'cluster {cluster2}']])[0]
            lista_kruskal_quant.append(resultado[0])
            lista_kruskal_quali.append(resultado[1])

    df_saida['Gamma de Kruskal - Quantitativo'] = lista_kruskal_quant
    df_saida['Gamma de Kruskal - Qualitativo'] = lista_kruskal_quali
    
    return df_saida

def analisa_genero(df):
    n_colunas = df.shape[1]
    
    df_saida = pd.DataFrame()

    lista = ['Todos os Clusters']
    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):
            lista.append(f'Cluster {cluster1} e {cluster2}')
    df_saida['SEXO'] = lista
    
    df_saida = aplica_cramer(df, df_saida)
    
    df_saida = aplica_yule(df, df_saida)
    
    df_saida = aplica_kruskal(df, df_saida)
    
    return df_saida

def analisa_idade(df):
    n_colunas = df.shape[1]
    
    df_saida = pd.DataFrame()

    lista = ['Todos os Clusters']
    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):
            lista.append(f'Cluster {cluster1} e {cluster2}')
    
    df_saida['IDADE'] = lista
    
    df_saida = aplica_kruskal(df, df_saida)
    
    return df_saida


def analisa_imc(df):
    n_colunas = df.shape[1]
    
    df_saida = pd.DataFrame()

    lista = ['Todos os Clusters']
    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):
            lista.append(f'Cluster {cluster1} e {cluster2}')
    
    df_saida['IMC'] = lista
    
    df_saida = aplica_kruskal(df, df_saida)
    
    return df_saida

def analisa_doencas(df):
    n_colunas = df.shape[1]
    
    df_saida = pd.DataFrame()

    lista = ['Todos os Clusters']
    for cluster1 in range(1, n_colunas + 1):
        for cluster2 in range(cluster1 + 1, n_colunas + 1):
            lista.append(f'Cluster {cluster1} e {cluster2}')
    
    df_saida['PADRÕES'] = lista
    
    df_saida = aplica_cramer(df, df_saida)
    
    return df_saida

def analisa_doenca_pares(df):
    lista_analise = list()
    
    doencas = list(df.index)
    
    for doenca1 in range(len(doencas)):
        for doenca2 in range(doenca1 + 1, len(doencas)):
            df_aux = df.loc[[doencas[doenca1], doencas[doenca2]], :]
            
            n_colunas = df_aux.shape[1]
    
            df_saida = pd.DataFrame()

            lista = ['Todos os Clusters']
            for cluster1 in range(1, n_colunas + 1):
                for cluster2 in range(cluster1 + 1, n_colunas + 1):
                    lista.append(f'Cluster {cluster1} e {cluster2}')

            df_saida[doencas[doenca1] + '/' + doencas[doenca2]] = lista
    
            df_saida = aplica_cramer(df_aux, df_saida)
            df_saida = aplica_yule(df_aux, df_saida)
            
            lista_analise.append(df_saida)
        
    return lista_analise
            
def salva_doenca_pares(df, lista):
    doencas = list(df.index)
    
    i = 0
    
    for doenca1 in range(len(doencas)):
        for doenca2 in range(doenca1 + 1, len(doencas)):
            nome = doencas[doenca1].replace(' ','_') + '&' +  doencas[doenca2].replace(' ','_')
            lista[i].to_csv(DIR_BASE / f"correlacao{lista_onde_salva[abordagem]}" / f"{algoritmo.upper()}{lista_onde_salva[abordagem]}_analise_doença_{nome}.csv", 
                           sep=";", encoding="latin1", index=False)
            i += 1

In [5]:
#caminho = DIRETORIO_PRINCIPAL / DIR_KMEANS
#caminho
#diretorios = {'dict':DIR_DICT,'kmeans':DIR_KMEANS, 'nmf':DIR_NMF}

nome_abordagem = {'cadastro':"_qtd_approach_cadastro", 'doenca':"_qtd_approach_doenca",
                  'medidas':"_qtd_approach_medidas",'estatistica':"_qtd_approach_estatistica",
                 'escolho_doenca':"_qtd_approach_escolho_doenca", 'categorias':"_qtd_approach_categorias"}

In [6]:
#DIR_DICT / f"Dict_approach_doenca
#_grupo_{grupo}.csv", 

# Análise é uma opção do dicionário criado na célula acima
algoritmo = input("Informe a análise ")

#KMEANS_qtd_approach_categorias.csv
caminho_medidas = DIR_BASE / f"{algoritmo.upper()}{nome_abordagem['medidas']}.csv"
caminho_estatistica = DIR_BASE / f"{algoritmo.upper()}{nome_abordagem['estatistica']}.csv"
caminho_doenca = DIR_BASE / f"{algoritmo.upper( )}{nome_abordagem['doenca']}.csv"
caminho_cadastro = DIR_BASE / f"{algoritmo.upper()}{nome_abordagem['cadastro']}.csv"
caminho_escolho_doenca = DIR_BASE / f"{algoritmo.upper()}{nome_abordagem['escolho_doenca']}.csv"
caminho_categorias = DIR_BASE / f"{algoritmo.upper()}{nome_abordagem['categorias']}.csv"

lista_caminhos = [caminho_medidas, caminho_doenca, caminho_cadastro, caminho_categorias]

lista_onde_salva = ['_medidas', '_doenca','_cadastro','_categorias']


Informe a análise kmeans


In [7]:
for abordagem in range(len(lista_caminhos)):
    df_abordagem = pd.read_csv(lista_caminhos[abordagem], sep=";", encoding='latin1')
    df_abordagem.index = df_abordagem['Cluster'].values
    df_abordagem = df_abordagem.drop(['Cluster'], axis = 1)
    
    try:
        df_genero = df_abordagem.loc[['Masculino','Feminino'], :]
        saida_genero = analisa_genero(df_genero)
        #  grava arquivo da abordagem por GÊNERO

        saida_genero.to_csv(DIR_BASE / f"correlacao{lista_onde_salva[abordagem]}" / f"{algoritmo.upper()}{lista_onde_salva[abordagem]}_analise_sexo.csv", 
                                   sep=";", encoding="latin1", index=False)
    except:
        pass
    
    try:
        df_idade = df_abordagem.loc[['Adultos','Meia-idade','Idoso'], :]
        #df_idade = df_abordagem.loc[['Jovens','Adultos','Meia-idade','Idoso','Ancião','Velhice extrema'], :]
        saida_idade = analisa_idade(df_idade)

        #  grava arquivo da abordagem por IDADE

        saida_idade.to_csv(DIR_BASE /  f"correlacao{lista_onde_salva[abordagem]}" / f"{algoritmo.upper()}{lista_onde_salva[abordagem]}_analise_idade.csv", 
                                   sep=";", encoding="latin1", index=False)
    except:
        pass
    
    try:
        df_imc = df_abordagem.loc[['Peso Normal','Sobrepeso','Obesidade'], :]
        #df_imc = df_abordagem.loc[['Abaixo do peso','Peso Normal','Sobrepeso','Obesidade grau 1','Obesidade grau 2','Obesidade Grau 3'], :]
        saida_imc = analisa_imc(df_imc)

        #  grava arquivo da abordagem por IMC

        saida_imc.to_csv(DIR_BASE / f"correlacao{lista_onde_salva[abordagem]}" / f"{algoritmo.upper()}{lista_onde_salva[abordagem]}_analise_imc.csv", 
                                   sep=";", encoding="latin1", index=False)
    except:
        pass
    
    try:
        df_doencas = df_abordagem.loc[['Sistolica Isolada','Diastolica Isolada','Dipping','Non Dipping','Extreme Dipping','Reverse Dipping','Morning Surge','Masked','Whitecoat'], :]
        
        saida_doencas = analisa_doencas(df_doencas)

        #  grava arquivo da abordagem por PADRÂO DE DOENÇA

        saida_doencas.to_csv(DIR_BASE / f"correlacao{lista_onde_salva[abordagem]}" / f"{algoritmo.upper()}{lista_onde_salva[abordagem]}_analise_doenca.csv", 
                                   sep=";", encoding="latin1", index=False)
        
        lista_analises = analisa_doenca_pares(df_doencas)
        salva_doenca_pares(df_doencas, lista_analises)
        
    except:
        pass

In [8]:
#df_genero = df_categorias.loc[['Masculino','Feminino'], :]
#df_genero


In [9]:
#df_idade = df_categorias.loc[['Jovens','Adultos','Meia-idade','Idoso','Ancião','Velhice extrema'], :]
#df_idade


In [10]:
#df_doencas = df_categorias.loc[['Sistolica Isolada','Diastolica Isolada','Dipping','Non Dipping','Extreme Dipping','Reverse Dipping','Morning Surge','Masked','Whitecoat'], :]
#df_doencas


In [11]:
#df_imc = df_categorias.loc[['Abaixo do peso','Peso Normal','Sobrepeso','Obesidade grau 1','Obesidade grau 2','Obesidade Grau 3'], :]
#df_imc

### Análise por GÊNERO

In [12]:
#saida_genero = analisa_genero(df_genero)
#saida_genero
#
##  grava arquivo da abordagem por GÊNERO
#
#saida_genero.to_csv(DIR_BASE / "correlacao_categorias" / f"{algoritmo.upper()}_analise_sexo.csv", 
#                           sep=";", encoding="latin1", index=False)

### Análise por IDADE

In [13]:
#saida_idade = analisa_idade(df_idade)
#saida_idade
#
##  grava arquivo da abordagem por IDADE
#
#saida_idade.to_csv(DIR_BASE / "correlacao_categorias" / f"{algoritmo.upper()}c_analise_idade.csv", 
#                           sep=";", encoding="latin1", index=False)

### Análise por IMC

In [14]:
#saida_imc = analisa_imc(df_imc)
#saida_imc
#
##  grava arquivo da abordagem por IMC
#
#saida_imc.to_csv(DIR_BASE / "correlacao_categorias" / f"{algoritmo.upper()}_analise_imc.csv", 
#                           sep=";", encoding="latin1", index=False)

### Análise por DOENÇA

In [15]:
#saida_doencas = analisa_doencas(df_doencas)
#saida_doencas
#
##  grava arquivo da abordagem por PADRÂO DE DOENÇA
#
#saida_doencas.to_csv(DIR_BASE / "correlacao_categorias" / f"{algoritmo.upper()}_analise_doenca.csv", 
#                           sep=";", encoding="latin1", index=False)

### Análise por DOENÇA - Pares

In [16]:
#lista_analises = analisa_doenca_pares(df_doencas)
#salva_doenca_pares(df_doencas, lista_analises)
#lista_analises[0]
#